# 📈🎵 株価データをMIDIに変換するMaxPyノートブック

このノートブックは以下の機能を提供します：

1. **株価データの取得** - yfinanceを使用して任意のtickerの株価を取得
2. **データのMIDI変換** - 株価の変動をMIDIノート、ベロシティ、デュレーションにマッピング
3. **MaxパッチとMIDIファイル生成** - MaxPyで再生可能なMaxパッチとMIDIファイルを作成
4. **ダウンロード** - 生成されたファイルをダウンロード

作成者: Claude + Akira  
日付: 2026年2月1日

## 📦 Step 1: 必要なライブラリのインストール

In [ ]:
# 必要なライブラリをインストール
!pip install -q maxpylang yfinance mido pandas matplotlib

print("✓ すべてのライブラリがインストールされました！")

## 📊 Step 2: 株価データの取得

In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# ===== パラメータ設定 =====
TICKER = "AAPL"  # 株式ティッカー（例: AAPL, GOOGL, TSLA, 7203.T など）
PERIOD = "1mo"   # 取得期間（1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max）
INTERVAL = "1d"  # データ間隔（1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo）

# 株価データを取得
print(f"📈 {TICKER} の株価データを取得中...")
stock = yf.Ticker(TICKER)
df = stock.history(period=PERIOD, interval=INTERVAL)

print(f"✓ {len(df)} 件のデータを取得しました")
print(f"  期間: {df.index[0].strftime('%Y-%m-%d')} ～ {df.index[-1].strftime('%Y-%m-%d')}")
print(f"\n最初の5件:")
print(df.head())

# データの可視化
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plt.plot(df.index, df['Close'], label='終値', color='blue', linewidth=2)
plt.title(f'{TICKER} 株価推移', fontsize=14, fontweight='bold')
plt.ylabel('価格 (USD)', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 1, 2)
plt.bar(df.index, df['Volume'], label='出来高', color='green', alpha=0.6)
plt.ylabel('出来高', fontsize=12)
plt.xlabel('日付', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('stock_chart.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ チャートを保存しました: stock_chart.png")

## 🎵 Step 3: 株価データをMIDIパラメータに変換

In [ ]:
import numpy as np

# ===== MIDI変換パラメータ =====
MIN_NOTE = 36      # 最低音（C1）
MAX_NOTE = 84      # 最高音（C6）
BASE_VELOCITY = 64 # 基本ベロシティ
NOTE_DURATION = 500 # ノートの長さ（ミリ秒）

def normalize_to_range(data, min_val, max_val):
    """データを指定範囲に正規化"""
    data_min = data.min()
    data_max = data.max()
    if data_max == data_min:
        return np.full_like(data, int((min_val + max_val) / 2), dtype=int)
    normalized = (data - data_min) / (data_max - data_min)
    return (normalized * (max_val - min_val) + min_val).astype(int)

# 終値をMIDIノート番号に変換
midi_notes = normalize_to_range(df['Close'].values, MIN_NOTE, MAX_NOTE)

# 価格変動率をベロシティに変換
price_change = df['Close'].pct_change().fillna(0).abs()
velocities = normalize_to_range(price_change.values * 100, 40, 127)

# 出来高をデュレーションに変換（出来高が多いほど長い音）
durations = normalize_to_range(df['Volume'].values, 200, 1000)

# データフレームに追加
df['MIDI_Note'] = midi_notes
df['Velocity'] = velocities
df['Duration_ms'] = durations

print("🎵 株価データをMIDIパラメータに変換しました！")
print(f"\nMIDIデータサンプル:")
print(df[['Close', 'Volume', 'MIDI_Note', 'Velocity', 'Duration_ms']].head(10))

# MIDI変換結果の可視化
plt.figure(figsize=(12, 8))

plt.subplot(3, 1, 1)
plt.plot(df.index, df['MIDI_Note'], marker='o', linestyle='-', color='purple')
plt.title('MIDIノート番号（株価に基づく）', fontsize=12, fontweight='bold')
plt.ylabel('ノート番号', fontsize=10)
plt.grid(True, alpha=0.3)

plt.subplot(3, 1, 2)
plt.plot(df.index, df['Velocity'], marker='s', linestyle='-', color='orange')
plt.title('ベロシティ（価格変動率に基づく）', fontsize=12, fontweight='bold')
plt.ylabel('ベロシティ', fontsize=10)
plt.grid(True, alpha=0.3)

plt.subplot(3, 1, 3)
plt.plot(df.index, df['Duration_ms'], marker='^', linestyle='-', color='green')
plt.title('デュレーション（出来高に基づく）', fontsize=12, fontweight='bold')
plt.ylabel('長さ (ms)', fontsize=10)
plt.xlabel('日付', fontsize=10)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('midi_parameters.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ MIDIパラメータチャートを保存しました: midi_parameters.png")

## 🎹 Step 4: MIDIファイルの生成

In [ ]:
from mido import Message, MidiFile, MidiTrack, MetaMessage

# MIDIファイルを作成
midi_file = MidiFile()
track = MidiTrack()
midi_file.tracks.append(track)

# トラック名を設定
track.append(MetaMessage('track_name', name=f'{TICKER} Stock Sonification', time=0))

# テンポを設定（BPM 120）
track.append(MetaMessage('set_tempo', tempo=500000, time=0))

# 各データポイントをMIDIノートに変換
time_delta = 480  # ノート間の時間（ticksで指定）

prev_duration = 0
for idx, (i, row) in enumerate(df.iterrows()):
    note = int(row['MIDI_Note'])
    velocity = int(row['Velocity'])
    duration = int(row['Duration_ms'] * 0.48)  # msをMIDI ticksに変換（概算）
    
    # Note On（最初のノートは即座に開始、以降は前のノートとの間隔を調整）
    if idx == 0:
        note_on_time = 0
    else:
        note_on_time = max(0, time_delta - prev_duration)
    
    track.append(Message('note_on', note=note, velocity=velocity, time=note_on_time))
    
    # Note Off
    track.append(Message('note_off', note=note, velocity=0, time=duration))
    
    prev_duration = duration

# MIDIファイルを保存
midi_filename = f'{TICKER}_stock_sonification.mid'
midi_file.save(midi_filename)

print(f"✓ MIDIファイルを生成しました: {midi_filename}")
print(f"  トラック数: {len(midi_file.tracks)}")
print(f"  ノート数: {len([msg for msg in track if msg.type == 'note_on'])}")
print(f"  総再生時間: {midi_file.length:.2f} 秒")

## 🎛️ Step 5: MaxPyでMax for Liveデバイス(.amxd)を生成

In [ ]:
import maxpylang as mp

# 新しいMax for Liveデバイスを作成
patch = mp.MaxPatch()

print("🎛️ Max for Liveデバイスを構築中...")

# MIDIファイル再生用のオブジェクトを配置
# Row 0: Max for Live特有のインレット/アウトレット
plugin_in = patch.place('plugin~')[0]
plugin_out = patch.place('plugout~')[0]

# Row 1: Live.path とファイル管理
live_path = patch.place('live.path')[0]
message_read = patch.place(f'message read {midi_filename}')[0]
message_1 = patch.place('message 1')[0]

# Row 2: MIDIシーケンサー
seq = patch.place('seq')[0]
message_start = patch.place('message start')[0]
message_stop = patch.place('message stop')[0]
message_bang = patch.place('button')[0]

# Row 3: MIDI処理
midiparse = patch.place('midiparse')[0]
stripnote = patch.place('stripnote')[0]

# Row 4: シンセサイザー（複数の音色）
makenote = patch.place('makenote 64 500')[0]

# サイン波オシレーター
mtof_1 = patch.place('mtof')[0]
cycle_1 = patch.place('cycle~')[0]
mult_1 = patch.place('*~ 0.3')[0]

# 矩形波オシレーター（倍音豊か）
mtof_2 = patch.place('mtof')[0]
rect = patch.place('rect~ 0.5')[0]
mult_2 = patch.place('*~ 0.2')[0]

# Row 5: Live専用エフェクト
# live.filterを使用
live_filter = patch.place('live.filter~')[0]

# Row 6: ミキサーと出力
mixer = patch.place('+~')[0]
live_gain = patch.place('live.gain~')[0]

# Row 7: 視覚化（Live専用）
live_scope = patch.place('live.scope~')[0]
live_meter = patch.place('live.meter~')[0]

# Row 8: 情報表示とパラメータ
live_text = patch.place(f'live.text @text "{TICKER} Stock"')[0]
live_numbox = patch.place('live.numbox')[0]
comment_info = patch.place(f'comment Period: {PERIOD}, Notes: {len(df)}')[0]

print("🔗 オブジェクトを接続中...")

# 接続: Live.path → MIDIファイル読み込み
patch.connect([live_path.outs[0], message_read.ins[0]])
patch.connect([message_read.outs[0], seq.ins[0]])

# 接続: コントロール → シーケンサー
patch.connect([message_start.outs[0], seq.ins[0]])
patch.connect([message_stop.outs[0], seq.ins[0]])
patch.connect([message_1.outs[0], seq.ins[0]])
patch.connect([message_bang.outs[0], seq.ins[0]])

# 接続: シーケンサー → MIDI処理
patch.connect([seq.outs[0], midiparse.ins[0]])
patch.connect([midiparse.outs[0], stripnote.ins[0]])
patch.connect([midiparse.outs[1], stripnote.ins[1]])

# 接続: MIDI → シンセ1（サイン波）
patch.connect([stripnote.outs[0], makenote.ins[0]])
patch.connect([stripnote.outs[1], makenote.ins[1]])
patch.connect([makenote.outs[0], mtof_1.ins[0]])
patch.connect([mtof_1.outs[0], cycle_1.ins[0]])
patch.connect([cycle_1.outs[0], mult_1.ins[0]])

# 接続: MIDI → シンセ2（矩形波）
patch.connect([makenote.outs[0], mtof_2.ins[0]])
patch.connect([mtof_2.outs[0], rect.ins[0]])
patch.connect([rect.outs[0], mult_2.ins[0]])

# 接続: シンセ → ミキサー
patch.connect([mult_1.outs[0], mixer.ins[0]])
patch.connect([mult_2.outs[0], mixer.ins[1]])

# 接続: ミキサー → フィルター → ゲイン（両チャンネルにモノ信号を送る）
patch.connect([mixer.outs[0], live_filter.ins[0]])
patch.connect([mixer.outs[0], live_filter.ins[1]])
patch.connect([live_filter.outs[0], live_gain.ins[0]])
patch.connect([live_filter.outs[1], live_gain.ins[1]])

# 接続: ゲイン → plugin出力（Live経由）
patch.connect([live_gain.outs[0], plugin_out.ins[0]])
patch.connect([live_gain.outs[1], plugin_out.ins[1]])

# 接続: 視覚化
patch.connect([live_gain.outs[0], live_scope.ins[0]])
patch.connect([live_gain.outs[0], live_meter.ins[0]])

# Max for Liveデバイスとして保存（.amxd形式）
amxd_filename = f'{TICKER}_stock_sonification.amxd'
patch.save(amxd_filename)

print(f"\n✓ Max for Liveデバイスを生成しました: {amxd_filename}")
print(f"  配置したオブジェクト数: {len(patch.boxes)}")
print(f"  接続数: {len(patch.lines)}")
print(f"\n💡 このデバイスはAbleton Liveで直接開けます！")

## 📊 Step 6: データ統計とサマリー

In [ ]:
print("="*60)
print(f"  {TICKER} 株価ソニフィケーション - サマリーレポート")
print("="*60)

print(f"\n📈 株価データ:")
print(f"  ティッカー: {TICKER}")
print(f"  期間: {df.index[0].strftime('%Y-%m-%d')} ～ {df.index[-1].strftime('%Y-%m-%d')}")
print(f"  データポイント数: {len(df)}")
print(f"  最高値: ${df['Close'].max():.2f}")
print(f"  最安値: ${df['Close'].min():.2f}")
print(f"  平均値: ${df['Close'].mean():.2f}")
print(f"  価格変動幅: ${df['Close'].max() - df['Close'].min():.2f}")

print(f"\n🎵 MIDIデータ:")
print(f"  ノート範囲: {df['MIDI_Note'].min()} ～ {df['MIDI_Note'].max()}")
print(f"  平均ノート: {df['MIDI_Note'].mean():.1f}")
print(f"  ベロシティ範囲: {df['Velocity'].min()} ～ {df['Velocity'].max()}")
print(f"  平均ベロシティ: {df['Velocity'].mean():.1f}")
print(f"  デュレーション範囲: {df['Duration_ms'].min()}ms ～ {df['Duration_ms'].max()}ms")

print(f"\n📁 生成されたファイル:")
print(f"  1. {midi_filename} - MIDIファイル")
print(f"  2. {amxd_filename} - Max for Liveデバイス")
print(f"  3. stock_chart.png - 株価チャート")
print(f"  4. midi_parameters.png - MIDIパラメータ可視化")

print("\n💡 使い方:")
print("  1. 下のセルを実行してファイルをダウンロード")
print("  2. Ableton Liveで.amxdファイルをダブルクリック")
print("  3. またはLiveのブラウザからデバイスをトラックにドラッグ")
print("  4. MIDIファイルが同じフォルダにあることを確認")
print("  5. 'start'ボタンをクリックして株価の音楽を再生")

print("\n" + "="*60)

## 💾 Step 7: ファイルのダウンロード

In [ ]:
from google.colab import files
import zipfile
import os

# ZIPファイルを作成
zip_filename = f'{TICKER}_stock_sonification_package.zip'

with zipfile.ZipFile(zip_filename, 'w') as zipf:
    # MIDIファイル
    if os.path.exists(midi_filename):
        zipf.write(midi_filename)
    
    # Max for Liveデバイス
    if os.path.exists(amxd_filename):
        zipf.write(amxd_filename)
    
    # チャート画像
    if os.path.exists('stock_chart.png'):
        zipf.write('stock_chart.png')
    
    if os.path.exists('midi_parameters.png'):
        zipf.write('midi_parameters.png')
    
    # CSVデータ
    csv_filename = f'{TICKER}_data.csv'
    df.to_csv(csv_filename)
    zipf.write(csv_filename)

print(f"✓ すべてのファイルを {zip_filename} にパッケージしました")
print("\nダウンロード中...")

# ZIPファイルをダウンロード
files.download(zip_filename)

print("\n🎉 完了！ファイルがダウンロードされました！")
print("\nAbleton Liveで.amxdファイルを開いて、株価の音楽をお楽しみください！")
print("Max for Liveデバイスとして、Liveのトラックに直接ドラッグ&ドロップできます！")

## 🎨 おまけ: カスタマイズのヒント

### パラメータを変更して実験してみましょう！

**Step 2のパラメータ:**
```python
TICKER = "TSLA"     # テスラの株価を試す
PERIOD = "6mo"      # 6ヶ月分のデータを取得
INTERVAL = "1d"     # 1日ごとのデータ
```

**Step 3のMIDI変換パラメータ:**
```python
MIN_NOTE = 48       # より高い音域（C2）
MAX_NOTE = 96       # より高い音域（C7）
NOTE_DURATION = 1000 # より長い音符
```

### 日本株を試すには:
```python
TICKER = "7203.T"   # トヨタ自動車
TICKER = "9984.T"   # ソフトバンクグループ
TICKER = "6758.T"   # ソニーグループ
```

### 異なるマッピングを試す:
- 出来高をベロシティに
- 価格変動率をデュレーションに
- 複数の指標を組み合わせる

---

**ハッピーソニフィケーション！** 🎵📈